<center>
<h1>
<h1>APM 53674: ALTeGraD</h1>
<h2>Lab Session 3: Improving LLMs with RLHF (DPO & GRPO)</h2>
<h4>Lecture: Prof. Michalis Vazirgiannis<br>
Lab: Yang Zhang and Xiao Fei</h4>
<h5>Tuesday, October 14, 2025</h5>
<br>
</center>

<hr style="border:10px solid gray"> </hr>
<p style="text-align: justify;">
This handout includes theoretical introductions, <font color='blue'>coding tasks</font> and <font color='red'>questions</font>. Before the deadline, you should submit <a href='https://forms.gle/9dyaes6dimfvyjwq6' target="_blank">here</a> a <B>.ipynb</B> file named <b>Lastname_Firstname.ipynb</b> containing your notebook (with the gaps filled and your answers to the questions). Your answers should be well constructed and well justified. They should not repeat the question or generalities in the handout. When relevant, you are welcome to include figures, equations and tables derived from your own computations, theoretical proofs or qualitative explanations. One submission is required for each student. The deadline for this lab is <b>October 19
, 2025 11:59 PM</b>. No extension will be granted. Late policy is as follows: ]0, 24] hours late → -5 pts; ]24, 48] hours late → -10 pts; > 48 hours late → not graded (zero).
</p>
<hr style="border:5px solid gray"> </hr>


# 🤖 Post-Training: Improving LLMs with RLHF (DPO & GRPO)

In this notebook, we’ll show how to improve a language model using **two post-training techniques**:

### 🧠 What You’ll Learn
- What **Direct Preference Optimization (DPO)** is and how it helps models choose better answers.
- What **Group Relative Policy Optimization (GRPO)** is and how it helps models solve tasks  improving reasoning and performance on complex tasks (math, code, logic).
- How to train small models on **real feedback data**.
- How to **observe changes in model behavior** after fine-tuning.


### 📦 What We’ll Use
- **Hugging Face 🤗 Transformers** to load and run models
- **TRL (Transformer Reinforcement Learning)** library by Hugging Face 🤗 for DPO and GRPO
- **A small version** of the French translated [Anthropic HH-RLHF dataset](https://huggingface.co/datasets/AIffl/french_hh_rlhf)
- **Colab GPU**, so models are small enough to run quickly

### Useful links:
- [Hugging Face 🤗 DPO Trainer](https://huggingface.co/docs/trl/dpo_trainer)
- [Hugging Face 🤗 GRPO Trainer](https://huggingface.co/docs/trl/grpo_trainer)

> This notebook is interactive, friendly, and high-level. You don’t need to know deep math or theory to follow along.

# Quick start:
1-  Clone the repository:
```bash
git clone https://github.com/BounharAbdelaziz/RLHF.git
```
2- Install the dependencies:
```bash
pip install -q -r requirements.txt
```
Now we are ready to go!

In [ ]:
!pwd

/content


In [ ]:
!git clone https://github.com/BounharAbdelaziz/RLHF.git

fatal: destination path 'RLHF' already exists and is not an empty directory.


In [ ]:
!pip install -q -r RLHF/requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 12.9 MB/s eta 0:00:00


# Part I: DPO

# 🧠 Fine-tuning Qwen2.5-0.5B-Instruct on French Data

In this part, we’ll walk through how to **fine-tune the Qwen2.5-0.5B-Instruct** model on **French-language data**, using **off-policy DPO (Direct Preference Optimization)**.

---

## 🧩 Key Concepts

- **Model**: [Qwen2.5-0.5B-Instruct](https://huggingface.co/Qwen/Qwen2.5-0.5B-Instruct)  
- **Objective**: Adapt the model for French understanding and instruction-following  
- **Method**: Off-policy **DPO** for alignment-based fine-tuning  

---

## ⚙️ System Requirements

Before training, make sure you choose an appropriate GPU setup.

- **Memory requirements**: [GPU memory guidance](https://rahulschand.github.io/gpu_poor/)  
- **GPU options**:
  - AWS SageMaker: [pricing calculator](https://aws.amazon.com/sagemaker-ai/pricing/)  
  - 💸 Cheaper alternative: [RunPod](https://www.runpod.io/)  

---

## 🧮 Memory Optimization

We’ll use **LoRA** (Low-Rank Adaptation) combined with **quantization (4-bit)** to reduce GPU memory usage while maintaining performance.

---

## 🧪 Evaluation Tools

To evaluate the fine-tuned model, you can use one or more of the following frameworks:

- [`lm-eval-harness`](https://github.com/EleutherAI/lm-evaluation-harness)
- [`lighteval`](https://github.com/huggingface/evaluate)
- [`libra-eval`](https://github.com/facebookresearch/libra)

---
Below is a visual overview of the **Direct Preference Optimization (DPO)** training process:

![DPO Training Diagram](https://1drv.ms/i/c/ae69638675180117/IQQ_wS77RKdrS4tzLbwoqr1gAR0Bf_1X2U36NRBp1Odsypg?width=560&height=48)

In [ ]:
# Imports
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
)
from trl import (
    DPOTrainer,
    DPOConfig,
)
from peft import (
    LoraConfig,
    get_peft_model,
    prepare_model_for_kbit_training,
)
import torch
import os
import wandb

## 📊 Tracking with Weights & Biases (W&B)

We’ll use **Weights & Biases (W&B)** to log training metrics, model versions, and system stats so you can compare runs, debug faster, and share results. Before running the next cell, **create a free account** at [https://wandb.ai](https://wandb.ai) and make a new **Project** (e.g., `RLHF`). In the code cell that follows, we’ll initialize W&B; on first use you’ll need to **register** then paste your **API key** from your W&B profile. During training, W&B will automatically track losses, learning rate, gradient norms, and GPU utilization, and we’ll log custom metrics (e.g., validation perplexity, evaluation scores) plus configuration details (dataset, hyperparameters, LoRA/quantization settings). Each run will appear on your project dashboard with charts, tables, run metadata, and artifacts, making it easy to **compare experiments**, **resume runs**, and **share dashboards** with your teammates.


<b><h4><font color='blue'>
<hr style="border:10px solid blue"> </hr>
Task 1: </b><br>
Create your Weights&Biases account and fill the gap the next cell.
<hr style="border:10px solid blue"> </hr>
</font></h4>

In [ ]:
# We will use wandb.ai for logging the experiments - Set your API key here
WANDB_API_KEY = "3eed34e1dab8e3cbcfc99646e97521cac9709865" # fill the gap with your wandb account

# Automatically login using the API key
os.environ["WANDB_API_KEY"] = WANDB_API_KEY
os.environ["WANDB_PROJECT"] = "RLHF"
wandb.login()

# Training dataset
DATASET_PATH = "AIffl/french_orca_dpo_pairs" # french version of "Intel/orca_dpo_pairs"

# We limit to 2k samples for speed
LIMIT = 2_000

# SFT Model we will finetune
MODEL_NAME = "Qwen/Qwen2.5-0.5B-Instruct"
# Seed for reproducibility
SEED = 1998

MAX_PROMPT_LEN = 1024
MAX_LENGTH = MAX_PROMPT_LEN + 512

RUN_NAME = "DPO-french-orca-" + MODEL_NAME.split('/')[-1]

## Load the SFT Model and Tokenizer

We’ll stick with 4-bit quantization via bitsandbytes for this lab. You’ve already used it last week, so nothing new—same setup (load the model with 4-bit weights), same goal (reduce VRAM) with minimal impact on quality for our use case. This keeps runs feasible on a single GPU.

We will need the tonkenizer in the data preparation step to apply the chat template.

<b><h4><font color='blue'>
<hr style="border:10px solid blue"> </hr>
Task 2: </b><br>
Create the quantization confiduration to load the model with 4 bits
<hr style="border:10px solid blue"> </hr>
</font></h4>

In [ ]:
# Quantization configuration
quantization_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype= torch.bfloat16, bnb_4bit_quant_type="nf4", bnb_4bit_use_double_quant=True)

# Load the model to finetune
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
    quantization_config=quantization_config,
    device_map="auto",
    trust_remote_code=True,
)
# to avoid warning
model.config.use_cache = False
# Prepare model for k-bit training
model = prepare_model_for_kbit_training(model)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)

# Set padding token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id

## Data Preparation

### 💬 Chat templates & converting data to `messages`

Modern instruction-tuned models (including **Qwen2.5-0.5B-Instruct**) expect inputs in a **chat format** and rely on a **tokenizer chat template** to turn structured messages into the exact token sequence the model was trained on. In practice, you should **not** hand-craft special tokens; instead, pass a list of `{role, content}` messages to the tokenizer and let `apply_chat_template(...)` do the right thing.

#### Why a chat template?
- Ensures your prompts match the **pretraining/finetuning format** (system/user/assistant turns, BOS/EOS, separators).
- Minimizes prompt drift across libraries and models.
- Makes it easy to add **system instructions** (e.g., “You are a helpful assistant that answers in French.”).

#### Message structure
Each example becomes an ordered list of chat turns:
```python
messages = [
  {"role": "system", "content": "Tu es un assistant utile. Réponds en français."},
  {"role": "user", "content": "Explique la différence entre LoRA et le fine-tuning complet."},
  {"role": "assistant", "content": "LoRA adapte un petit sous-espace de poids, alors que..."}
]


<b><h4><font color='blue'>
<hr style="border:10px solid blue"> </hr>
Task 3: </b><br>
Create the user message which is the question field of the dataset.
<hr style="border:10px solid blue"> </hr>
</font></h4>

In [ ]:
def preprocess_for_dpo(example):
    # Format system message if present
    messages = []
    if example.get('system') and len(example['system'].strip()) > 0:
        messages.append({"role": "system", "content": example['system']})

    # fill the gap, add user message
    user_message = {"role": "user", "content": example.get('question')}
    messages.append(user_message)

    # Create prompt with generation prompt for DPO
    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    # The chosen and rejected should be the assistant responses only
    chosen = example['chosen']
    rejected = example['rejected']

    return {
        "prompt": prompt,
        "chosen": chosen,
        "rejected": rejected,
    }

# Download the training dataset
dataset = load_dataset(DATASET_PATH, split=f"train")
# shuffle and select a number of samples
dataset = dataset.shuffle(True).select(range(LIMIT))

# Save columns
original_columns = dataset.column_names

# Apply the preprocessing function
dpo_dataset = dataset.map(
    preprocess_for_dpo,
    remove_columns=original_columns,
)

# Filter out examples that are too long
def filter_length(example):
    prompt_length = len(tokenizer.encode(example['prompt']))
    chosen_length = len(tokenizer.encode(example['chosen']))
    rejected_length = len(tokenizer.encode(example['rejected']))

    return (prompt_length + max(chosen_length, rejected_length)) < MAX_LENGTH

dpo_dataset = dpo_dataset.filter(filter_length)

print(f"Dataset size after filtering: {len(dpo_dataset)}")

In [ ]:
display(dpo_dataset.column_names)
display(dpo_dataset.shape)

## Model Training

Training will mirror last week’s lab, but we’ll switch from SFT to **off-policy DPO** using the `trl` library. Concretely, we’ll instantiate a **policy model** (trainable) and a **reference model** (frozen) and optimize with the DPO objective so the policy prefers **chosen** over **rejected** responses for the same prompt.

### What we’ll use
- **TRL**: `DPOConfig`, `DPOTrainer`
- **PEFT**: LoRA adapters on top of the base **Qwen2.5-0.5B-Instruct**
- **Quantization**: 4-bit (QLoRA-style) to fit on small GPUs
- **Logging**: W&B for metrics, configs, and artifacts

### Expected dataset columns
- `prompt` (or `messages`): the shared context (system+user turns)
- `chosen`: assistant reply preferred by annotators
- `rejected`: less-preferred reply
> If you’re keeping everything in chat format, we’ll pass lists of `{role, content}` and rely on `tokenizer.apply_chat_template(...)` inside the collator.

### Minimal training flow
1. Load tokenizer with the **chat template** and enable 4-bit loading of the base model.
2. Wrap the model with **LoRA** (target attention/MLP modules).
3. Build a `datasets.Dataset` that yields `(prompt/messages, chosen, rejected)`.
4. Define `DPOConfig` (batch size, lr, epochs, `beta`, logging/saving/eval cadence).
5. Create `DPOTrainer(policy_model, ref_model, tokenizer, train_dataset, eval_dataset, **config)`.
6. Call `trainer.train()`; optional `trainer.evaluate()` and `trainer.save_model()`.


<b><h4><font color='blue'>
<hr style="border:10px solid blue"> </hr>
Task 4: </b><br>
Create the lora config with rank of 32, alpha of 64 and dropout of 0.1 on all MLP layers (execluding Embedding layers) and train the model
<hr style="border:10px solid blue"> </hr>
</font></h4>

In [ ]:

# LoRA configuration - targeting the correct modules for Qwen2.5
peft_config = LoraConfig(r=32, lora_alpha=64, lora_dropout=0.1, target_modules= ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"], bias="none", task_type = 'CAUSAL_LM')

# Apply LoRA to the model
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

# Training configuration
training_args = DPOConfig(
    beta=0.1,  # DPO temperature parameter
    learning_rate=5e-6,
    max_prompt_length=MAX_PROMPT_LEN,
    max_length=MAX_LENGTH,
    per_device_train_batch_size=1,  # Reduced for memory
    gradient_accumulation_steps=4,  # Increased to maintain effective batch size of 4 (1*4)
    num_train_epochs=1,
    max_grad_norm=1.0,
    logging_steps=1,
    save_steps=100,
    lr_scheduler_type="cosine",
    optim="paged_adamw_8bit",  # More memory efficient
    warmup_ratio=0.03, # 3% of the steps will be just a warmup
    save_strategy="steps",
    output_dir="./dpo_model",
    report_to="wandb",
    run_name=RUN_NAME,
    remove_unused_columns=False,
    dataloader_pin_memory=False,
    fp16=True,  # Enable mixed precision
)

# Initialize the trainer - Note: no ref_model needed when using peft_config
trainer = DPOTrainer(
    model=model,
    args=training_args,
    peft_config=peft_config,  # This automatically handles reference model
    processing_class=tokenizer,
    train_dataset=dpo_dataset,
)

# Print a sample to verify preprocessing
print("Sample from dataset:")
print(f"Prompt: {dpo_dataset[0]['prompt']}")
print(f"Chosen: {dpo_dataset[0]['chosen']}")
print(f"Rejected: {dpo_dataset[0]['rejected']}")

# Train
trainer.train()

In [ ]:
# merge LoRA adapters with the base model
save_path = "dpo_model/final_merged_dpo_model"

model = model.merge_and_unload()
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

## Model Testing

We will test the DPO model via chat-app
To validate alignment gains, we’ll spin up a small **Gradio** app that queries both the **pre-DPO** model (baseline/reference) and the **post-DPO** **policy** side-by-side. The UI lets you enter a French prompt, then compares generations using the **same chat template** and decoding settings. This helps spot qualitative shifts in helpfulness, safety, and instruction-following.

In [ ]:
from RLHF.chat_app import launch_chat_app

launch_chat_app(
    model_path=save_path, #"habdine/CSC_53432_lab3_dpo",
    base_model_path="Qwen/Qwen2.5-0.5B-Instruct",
    title="🤖 Dual-Model Qwen Chat (DPO vs Base) for French",
    DPO_TEST=True,
    FRENCH_TEST=True,
)

In [ ]:
from RLHF.chat_app import launch_chat_app

launch_chat_app(
    model_path="BounharAbdelaziz/Qwen2.5-0.5B-DPO-English-Orca",
    base_model_path="Qwen/Qwen2.5-0.5B-Instruct",
    title="🤖 Dual-Model Qwen Chat (DPO vs Base) for English",
    DPO_TEST=True,
    FRENCH_TEST=False,
)

# Part II: GRPO


The following diagram illustrates the **GRPO (Generative Reinforcement Preference Optimization)** process — an alternative to DPO that directly optimizes generation quality from preference data using reinforcement-style updates:

![GRPO Training Overview](https://1drv.ms/i/c/ae69638675180117/IQQ-KizPdUxCRZU9qDGcpX1AAeettH1uhsJqqM1WjXiYR6s?width=705&height=66)

After exploring DPO, we now move on to **GRPO** — a reinforcement learning–style approach that builds directly on preference data.  
While **DPO** adjusts the model using an *analytic loss* derived from preference pairs, **GRPO** takes a more dynamic route: it uses **reward modeling and policy gradients** to optimize the model through sampled generations.

In essence:
- GRPO **learns from human (or model) preferences** using *on-policy* updates.  
- It combines elements of **PPO** (Proximal Policy Optimization) with **preference-based rewards** rather than explicit numerical scores.  
- This allows the model to better capture *generation quality* aspects that aren’t directly expressible through static loss terms.

In the next section, we’ll explore how to configure and launch a **GRPO training loop** using `trl`, reusing much of our previous setup (tokenizer, LoRA, quantization) but switching to **on-policy optimization**.

In [ ]:
import torch
import re
from datasets import load_dataset, Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
)
from peft import (
    LoraConfig,
    get_peft_model,
    TaskType,
    prepare_model_for_kbit_training,
)
from trl import (
    GRPOConfig,
    GRPOTrainer,
)
import os
import wandb

# We will use wandb.ai for logging the experiments - Set your API key here
WANDB_API_KEY = "3eed34e1dab8e3cbcfc99646e97521cac9709865" # fill the gap with your wandb key

# Automatically login using the API key
os.environ["WANDB_API_KEY"] = WANDB_API_KEY
os.environ["WANDB_PROJECT"] = "RLHF"
wandb.login()

# Training dataset
DATASET_PATH = "openai/gsm8k"

# We limit to 200 samples for speed
LIMIT = 200

# SFT Model we will finetune
MODEL_NAME = "Qwen/Qwen2.5-0.5B-Instruct"
# MODEL_NAME = "Qwen/Qwen2.5-3B-Instruct"
# Seed for reproducibility
SEED = 1998

USE_LORA = True

if MODEL_NAME == "Qwen/Qwen2.5-0.5B-Instruct":
  USE_QUANT = False
else:
  USE_QUANT = True

lora_alpha = 128
lora_r = 64
lora_dropout = 0.1

if not USE_LORA:
  MAX_PROMPT_LEN = 512
  MAX_LENGTH = MAX_PROMPT_LEN + 512
else:
    MAX_PROMPT_LEN = 150
    MAX_LENGTH = MAX_PROMPT_LEN + 150

RUN_NAME = "GRPO-GSM8K-limit-" + str(LIMIT) + "-" + MODEL_NAME.split('/')[-1]

## Load the SFT Model and Tokenizer

In [ ]:
# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Quantization configuration
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

# Load the model to finetune
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
    quantization_config=quantization_config if USE_QUANT else None,
    device_map="cuda:0",
    trust_remote_code=True,
)

# Prepare model for k-bit training
if USE_QUANT:
  model = prepare_model_for_kbit_training(model)
model.config.use_cache = False

# Add padding token if not exists
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

if USE_LORA:
  # Configure LoRA
  lora_config = LoraConfig(
      r=lora_r,  # Rank of adaptation : 64
      lora_alpha=lora_alpha,  # LoRA scaling parameter : 128
      target_modules=[
          "q_proj",
          "k_proj",
          "v_proj",
          "o_proj",
          "gate_proj",
          "up_proj",
          "down_proj",
      ],  # Target modules for Qwen2.5 architecture
      lora_dropout=lora_dropout,  # LoRA dropout : 0.1
      bias="none",  # Bias type
      task_type=TaskType.CAUSAL_LM,  # Task type
  )

  # Apply LoRA to the model
  model = get_peft_model(model, lora_config)

  # Print trainable parameters
  model.print_trainable_parameters()

## Data Preparation

For GRPO training, we’ll use the **GSM8K** dataset — a benchmark of grade-school math word problems.  
Each problem includes a **question** and a **final answer**. Our goal is to teach the model to reason in French (or English if you prefer) and **output only the final numeric answer** enclosed between `<answer>` and `</answer>` tags.

This format makes automatic evaluation trivial — we can extract the number between tags and compare it directly to the reference.

With GRPO the model will learn two main things:
- How follow the instruction to output the correct format.
- Gain more math capabilities

---

### 🧠 Why structure as chat messages?

Just like with DPO, the **Qwen2.5-0.5B-Instruct** model expects inputs in a *chat-style message format*.  
We’ll use:
- A **system message** to define the task and output style.
- A **user message** with the math problem.
- An **assistant message** containing the reasoning and final numeric answer wrapped in tags.

<b><h4><font color='blue'>
<hr style="border:10px solid blue"> </hr>
Task 5: </b><br>
Create your training samples as following:<br>
1- system message: R1_STYLE_SYSTEM_PROMPT + "\n" + TASK_SPECIFIC_INSTRUCTIONS <br>
2- one-shot example of user message of "What is 2+2?" and an assistant message of "To calculate 2+2, we simply add the numbers together: 2 + 2 = 4.\n$<answer>$4$</answer>$" <br>
3- the question sample from the dataset as a user message.
<hr style="border:10px solid blue"> </hr>
</font></h4>

In [ ]:
# Load GSM8K dataset
if LIMIT:
    dataset = load_dataset(DATASET_PATH, "main", split=f"train[:{LIMIT}]")  # Small subset for demo
else:
    dataset = load_dataset(DATASET_PATH, "main")

R1_STYLE_SYSTEM_PROMPT = """A conversation between User and Assistant. The user asks a question, and the Assistant solves it.
The assistant first thinks shortly about the reasoning process in the mind and then provides the user with the answer in a new line between <answer> and </answer>."""

TASK_SPECIFIC_INSTRUCTIONS = "The answer must be a single integer."
# print(dataset.column_names)
# print(dataset[0])
def preprocess_dataset(dataset, chunk_size=1000) -> Dataset:

    def extract_hash_answer(text: str) -> str | None:
        try:
            return text.split("####")[1].strip()
        except IndexError:
            return None

    def process_batch(batch):
        prompts = []
        for i in range(len(batch['question'])):
            messages = [
                {'role': 'system', 'content': R1_STYLE_SYSTEM_PROMPT + "\n" + TASK_SPECIFIC_INSTRUCTIONS},
                # One-shot example
                {'role': 'user', 'content': "What is 2+2?"},
                {'role': 'assistant', 'content': "To calculate 2+2, we simply add the numbers together: 2 + 2 = 4.\n<answer>4</answer>"},
                # Task-specific user message
                {'role': 'user', 'content': batch['question'][i].strip()}
            ]
            prompts.append(messages)

        return {
            'prompt': prompts,
            'answer': [extract_hash_answer(a) for a in batch['answer']]
        }

    return dataset.map(process_batch, batched=True, batch_size=chunk_size)
train_dataset = preprocess_dataset(dataset, chunk_size=500)

In [ ]:
print(train_dataset.column_names)

print(train_dataset[0]['answer'])
display(train_dataset[0])

## Reward Function Design (2)

We’ll use **two simple rewards** during GRPO rollouts:

1. **Format reward** — checks that the **last non-empty line** is exactly in the form  
   `<answer>NUMBER</answer>`  
   - Score: **1** if correct format, **0** otherwise.

2. **Correctness reward** — checks whether the extracted number matches the gold answer.  
   - Score: **2** if correct, **0** otherwise.

Total reward per sample ∈ {0, 1, 2, 3}.



<b><h4><font color='blue'>
<hr style="border:10px solid blue"> </hr>
Task 6: </b><br>
write the `extract_xml_answer` function to extract the answer from the generated text.
<hr style="border:10px solid blue"> </hr>
</font></h4>

In [ ]:
def extract_xml_answer(text: str) -> str:
    answer = text[text.find("<answer>")+len("<answer>"):text.find("</answer>")]
    return answer

def format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has the correct format."""
    pattern = r"^(?:[^\r\n]*\r?\n)+<answer>\d+</answer>\r?\n?$"
    responses = [completion[0]["content"] for completion in completions]
    matches = [bool(re.match(pattern, r)) for r in responses]
    return [1.0 if match else 0.0 for match in matches]

def correctness_reward_func(prompts, completions, answer, **kwargs) -> list[float]:
    """Reward function that checks if the answer is correct."""
    responses = [completion[0]['content'] for completion in completions]
    extracted_responses = [extract_xml_answer(r) for r in responses]
    return [2.0 if r == a else 0.0 for r, a in zip(extracted_responses, answer)]

## Model Training

In [ ]:
# GRPO Configuration
grpo_config = GRPOConfig(
    output_dir="./grpo_model",
    learning_rate=1e-5,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=1,
    num_train_epochs=3,
    max_prompt_length=MAX_PROMPT_LEN,
    max_completion_length=MAX_LENGTH,
    num_generations=2, # The effective train batch size must be evenly divisible by the number of generations per prompt
    beta=0,
    epsilon=0.28,
    temperature=1,
    logging_steps=1,
    save_steps=25,
    save_total_limit=3,
    # load_best_model_at_end=True,
    # metric_for_best_model="reward",
    # greater_is_better=True,
    run_name=RUN_NAME,
    lr_scheduler_type="cosine",
    optim="paged_adamw_8bit" ,  # More memory efficient
    warmup_ratio=0.03, # 3% of the steps will be just a warmup
    remove_unused_columns=False,
    dataloader_pin_memory=False,
    fp16=True,  # Enable mixed precision
)

# Initialize trainer
trainer = GRPOTrainer(
    model=model,
    reward_funcs=[format_reward_func, correctness_reward_func],
    args=grpo_config,
    train_dataset=train_dataset,
    processing_class=tokenizer,
)

# Training
print("Starting GRPO training...")
trainer.train()

In [ ]:
# merge LoRA adapters with the base model
save_path = "grpo_model/final_merged_grpo_model"

model = model.merge_and_unload()
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

## Model Testing

In [ ]:
from RLHF.chat_app import launch_chat_app

launch_chat_app(
    model_path=save_path,#"habdine/CSC_53432_lab3_grpo"
    base_model_path="Qwen/Qwen2.5-0.5B-Instruct",
    title="🤖 Dual-Model Qwen Chat (GRPO vs Base) for Math",
    DPO_TEST=False,
)

## 📈 Model Evaluation

Once our GRPO-trained model is ready, we need to **evaluate its performance**  — to verify that it has learned to produce correctly formatted and accurate answers.

For computational issues, we will evaluate on the first 200 samples only.


In [ ]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer
from tqdm.notebook import tqdm
import numpy as np
from typing import List, Dict
import json
from datetime import datetime
import logging


def extract_hash_answer(text: str) -> str | None:
    try:
        return text.split("####")[1].strip()
    except IndexError:
        return None

def evaluate_model(
    model_path: str,
    batch_size: int = 1,
    num_samples: int = None,
    save_results: bool = True,
) -> Dict:
    print("Initializing evaluation...")

    with tqdm(total=2, desc="Loading model components") as pbar:
        llm = AutoModelForCausalLM.from_pretrained(
            model_path,
            torch_dtype=torch.float16,
            device_map="cuda:0",
            trust_remote_code=True,
        )
        pbar.update(1)

        tokenizer = AutoTokenizer.from_pretrained(
            model_path,
            model_max_length=768,
        )
        pbar.update(1)

    # Load test dataset
    print("Loading dataset...")
    dataset = load_dataset('openai/gsm8k', 'main', split='test')
    if num_samples:
        dataset = dataset.select(range(num_samples))
    total_samples = len(dataset)
    print(f"Loaded {total_samples} samples")

    results = []
    correct = 0
    total = 0

    # Create progress bar
    progress_bar = tqdm(
        total=total_samples,
        desc="Processing samples",
        unit="examples",
        dynamic_ncols=True,
    )

    progress_bar.set_postfix({
        'acc': '0.00%',
        'correct': '0',
    })

    # Process in batches
    for i in range(0, total_samples, batch_size):
        batch_data = dataset[i:i + batch_size]
        current_batch_size = len(batch_data['question'])

        # Prepare prompts using same format as training
        prompts = [
            [
                {'role': 'system', 'content': R1_STYLE_SYSTEM_PROMPT + "\n" + TASK_SPECIFIC_INSTRUCTIONS},
                {'role': 'user', 'content': "What is 2+2?"},
                {'role': 'assistant', 'content': "To calculate 2+2, we simply add the numbers together: 2 + 2 = 4.\n<answer>4</answer>"},
                {'role': 'user', 'content': q.strip()}
            ] for q in batch_data['question']
        ]

        # Convert to chat format
        formatted_prompts = [
            tokenizer.apply_chat_template(
                p,
                tokenize=True,
                return_tensors='pt',
                add_generation_prompt=True
            )
            for p in prompts
        ]

        # Generate responses
        outputs = []
        for prompt in formatted_prompts:
            output = llm.generate(
                prompt.to('cuda:0'),
                max_new_tokens=512,
                temperature=1.0,
            )
            outputs.append(output)


        # Process responses
        for j, output in enumerate(outputs):

            response = tokenizer.decode(output[0], skip_special_tokens=True)

            # Extract answers
            generated_answer = extract_xml_answer(response)
            true_answer = extract_hash_answer(batch_data['answer'][j])

            # Store result
            result = {
                'question': batch_data['question'][j],
                'true_answer': true_answer,
                'generated_answer': generated_answer,
                'full_response': response,
                'correct': generated_answer == true_answer
            }
            results.append(result)

            # Update metrics
            if generated_answer == true_answer:
                correct += 1
            total += 1

        # Update progress
        progress_bar.update(current_batch_size)
        progress_bar.set_postfix({
            'acc': f'{(correct/total)*100:.2f}%',
            'correct': f'{correct}/{total}',
        })

    progress_bar.close()

    # Calculate metrics
    accuracy = correct / total if total > 0 else 0
    metrics = {
        'accuracy': accuracy,
        'correct': correct,
        'total': total,
        'model_path': model_path,
        'timestamp': datetime.now().isoformat()
    }

    # Save results
    if save_results:
        save_path = f"gsm8k_eval_results_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
        with open(save_path, 'w') as f:
            json.dump({
                'metrics': metrics,
                'results': results
            }, f, indent=2)
        print(f"\nResults saved to {save_path}")

    return metrics

print("Starting GSM8K evaluation...")


<b><h4><font color='blue'>
<hr style="border:10px solid blue"> </hr>
Task 7: </b><br>
evaluate the model before and after GRPO training.
<hr style="border:10px solid blue"> </hr>
</font></h4>

### Evaluation after GRPO



In [ ]:
grpo_file_path = "/content/grpo_model/final_merged_grpo_model"
metrics = evaluate_model(grpo_file_path, num_samples=LIMIT)



NameError: name 'evaluate_model' is not defined

In [ ]:
results_grpo ="/content/gsm8k_eval_results_20251018_164510.json"


### Evaluation before GRPO

In [ ]:
base_model = "Qwen/Qwen2.5-0.5B-Instruct"
evaluate_model(base_model, num_samples=LIMIT)

<b><h4><font color='red'>
<hr style="border:10px solid red"> </hr>
Question 1: </b><br>

- What are the main conceptual and practical differences between **Direct Preference Optimization (DPO)** and **Group Relative Policy Optimization (GRPO)**?  
- In what situations might one method outperform the other?  
- How do both approaches approximate the **KL-regularized RL objective** derived in [Christiano et al., 2017](https://arxiv.org/abs/1706.03741)?

📖 *References:*  
- Rafailov et al., 2023. *Direct Preference Optimization: Your Language Model is Secretly a Reward Model.* [arXiv:2305.18290](https://arxiv.org/abs/2305.18290)
<hr style="border:10px solid red"> </hr>
</font></h4>  


<b><h4><font color='green'>
<hr style="border:10px solid red"> </hr>
Answer 1: </b><br>

<hr style="border:10px solid red"> </hr>
</font></h4>

**1. GRPO overview**

**GRPO** is one of the existing methods for fine-tuning language models with human feedback (RLHF).

* Reward moddeling phase:  
First, it fits a reward model $r_ϕ$ to a dataset of prompts and
human preferences over pairs of responses. To do so, it uses a preference model such as the [Bradley–Terry model](https://en.wikipedia.org/wiki/Bradley%E2%80%93Terry_model#math_1).

* RL Fine-Tuning Phase:  
Then, it uses RL to find a policy that maximizes the learned reward. This phase aims to solve the following optimization problem: ![GRPO Training Overview](https://1drv.ms/i/c/ae69638675180117/IQQ-KizPdUxCRZU9qDGcpX1AAeettH1uhsJqqM1WjXiYR6s?width=705&height=66)



Conceptually it is close to RLHF with PPO but removes the critic and replaces advantage estimation with relative scoring across the groups.  

Practically, GRPO requires on-the-fly generation and a reward function, is more compute-intensive and potentially less stable, but it is flexible: rewards can be continuous, automatically computed, and adapted during training. Nevertheless, **fine-tuning large language models with reinforcement learning remains a major practical challenge** and that lead to approaches optimizing relative preferences without RL, such as **DPO**.


**2. DPO overview**

Unlike prior RLHF methods, and in particular **GRPO**, which learn a reward and then optimize it via RL, **DPO** leverages a particular choice of reward model parameterization that enables extraction of its optimal policy in closed form, **without an RL training loop**.
![DPO](https://1drv.ms/i/c/ae69638675180117/IQQ_wS77RKdrS4tzLbwoqr1gAR0Bf_1X2U36NRBp1Odsypg?width=560&height=48)     (1)


Theoretically, it also relies on a [Bradley–Terry model](https://en.wikipedia.org/wiki/Bradley%E2%80%93Terry_model#math_1) linking preferences to an implicit reward, practically, it is **simple, stable, data-efficient, and cheap to run**, but it is limited to pairwise supervision and cannot directly exploit graded or task-specific, automatically computable rewards.

**3. In what situations might one method outperform the other?**  
  
**DPO** is used when you have large, clean pairwise-preference datasets and limited compute, or when you want stable alignment with minimal policy drift. Also, it often when robustness and simplicity are key.

**GRPO** is used when you can define reliable automatic rewards and need online improvement—math/code/format-sensitive tasks, safety shaping with dense signals, non-binary preferences, or domains where exploration matters and targets evolve. Moreover, it outperforms **DPO** on verifiable tasks with informative rewards or when long, diverse generations benefit from relative scoring.

**4. Relation to the KL-regularized RL objective based on (Christiano et al., 2017) for each method.**

The KL-regularized RL objective is:

$\displaystyle \max_{\pi_\theta}\mathbb{E}_{x\sim D, y\sim \pi_\theta(\cdot\mid x)}\big[r(x,y)\big]-\beta \ \mathrm{KL}\big(\pi_\theta(\cdot\mid x)\ ||\  \pi_{\text{ref}}(\cdot\mid x)\big).$  

For **DPO**, starting from this RL objective, it is shown in [Rafailov et al., 2023.](https://arxiv.org/pdf/2305.18290#page=14&zoom=100,144,905) that the optimal solution is given in closed form by:

$$
\pi_r(y \mid x)
= \frac{1}{Z(x)}\, \pi_{\text{ref}}(y \mid x)\, \exp\!\left(\frac{1}{\beta}\, r(x,y)\right)
$$
where, $Z(x) = \sum_{y} \pi_{\text{ref}}(y \mid x)\,\exp\!\left(\frac{1}{\beta}\, r(x,y)\right)$

This gives an *implicit reward* $
\ r(x,y) = \beta \log \frac{\pi_r(y \mid x)}{\pi_{\text{ref}}(y \mid x)} + \beta \log Z(x)
$
, and after some steps a final loss:

$$
\mathcal{L}_{\text{DPO}}(\pi_\theta;\,\pi_{\text{ref}})
= -\,\mathbb{E}_{(x,y_w,y_l)\sim \mathcal{D}}
\bigg[
\log \sigma\!\Big(
\beta \log \frac{\pi_\theta(y_w \mid x)}{\pi_{\text{ref}}(y_w \mid x)}
-\beta \log \frac{\pi_\theta(y_l \mid x)}{\pi_{\text{ref}}(y_l \mid x)}
\Big)
\bigg].
$$  

  

In the other hand, **GRPO** directly forms a sampled estimator of the same objective by weighting $\nabla_\theta \log \pi_\theta(y_i\mid x)$ with softmax-normalized $r(x,y_i)$ across group samples and adding an explicit $\beta \ \mathrm{KL}\big(\pi_\theta \ || \ \pi_{\text{ref}}\big)$ regularizer.  

![GRPO Training Overview](https://1drv.ms/i/c/ae69638675180117/IQQ-KizPdUxCRZU9qDGcpX1AAeettH1uhsJqqM1WjXiYR6s?width=705&height=66)

<b><h4><font color='red'>
<hr style="border:10px solid red"> </hr>
Question 2: </b><br>

- Why can **reward model overfitting** or **reward hacking** occur in reinforcement fine-tuning?  
- How do DPO and GRPO attempt to mitigate this without explicit online reward models?  
- Discuss the role of the **reference model** in maintaining stability.
<hr style="border:10px solid red"> </hr>
</font></h4>

<b><h4><font color='green'>
<hr style="border:10px solid red"> </hr>
Answer 2: </b><br>

<hr style="border:10px solid red"> </hr>
</font></h4>

**1. Why reward model overfitting / reward hacking occurs.**  

In **RLHF** and in particular in **GRPO**, the reward model is trained on a small, biased set of preferences, so it can learn shallow cues instead of true quality. During RL, the policy explores regions the reward model never saw and finds blind spots that give high scores without better answers. Sparse or noisy rewards and weak KL control make this easier, and GRPO’s generation can amplify it by probing those blind spots faster.

**2. How DPO and GRPO mitigate this without explicit online reward models.**  

**DPO** avoids an online reward model. It turns pairwise preferences into a closed-form loss that compares the policy to a fixed reference (1). This directly optimizes relative preference gaps and keeps the policy anchored by a KL term, which limits drift into regions where a learned reward would be unreliable.

**GRPO** reduces reliance on a fragile reward by using group-relative scoring instead of absolute scores, plus a KL to a reference to cap distribution shift. It can use simple, automatic rewards and early stopping, which lowers the chance of chasing spurious patterns that a learned reward would amplify.

**3. Role of the reference model in maintaining stability.**  

In **GRPO**, the reference policy $\pi_{\text{ref}}$ is an anchor that enforces a trust region via an explicit or implicit KL term, typically maximizing $-\beta \ \mathrm{KL}\big(\pi_\theta(\cdot\mid x) \ ||\ \pi_{\text{ref}}(\cdot\mid x)\big)$ so updates keep $\pi_\theta$ in a high-likelihood neighborhood of a well-behaved base model.  

In **DPO**, $\pi_{\text{ref}}$ appears inside the log-ratio that defines the preference margin, acting as a built-in KL regularizer:
$$
\max_{\pi_\theta}\;
\mathbb{E}_{y\sim \pi_\theta(y\mid x)}
\Bigg[{r_\phi(x,y)
-\beta \log \sum_{y} \pi_{\text{ref}}(y\mid x)\,
\exp\!\left(\frac{1}{\beta}\, r_\phi(x,y)\right)}
\;-\;
\underbrace{\beta \log \frac{\pi_\theta(y\mid x)}{\pi_{\text{ref}}(y\mid x)}}_{\text{KL}}
\Bigg]
$$

The hyperparameter $\beta$ sets the trade-off between following the reward and staying close to the reference. Small $\beta$ means a strong KL pull to the reference, **safer and more stable but conservative and slower to improve.** Large $\beta$ weakens the KL, the model chases reward more, **learns faster, but risks drift and reward hacking.**

<b><h4><font color='red'>
<hr style="border:10px solid red"> </hr>
Question 3: </b><br>

- In the GSM8K setup, we used a simple binary reward for **format** and **correctness**.  
  - What are the limitations of such sparse rewards?  
  - How could you design a **richer, smoother** reward signal for math reasoning?
<hr style="border:10px solid red"> </hr>
</font></h4>

<b><h4><font color='green'>
<hr style="border:10px solid red"> </hr>
Answer 3: </b><br>

<hr style="border:10px solid red"> </hr>
</font></h4>

**Limiations of such sparse rewards**
1. No gradient for “near-misses”: A response that is numerically off by 1, or perfectly reasoned but with a tiny parsing mistake, gets the same 0 as a random guess, which is poor credit assignment and inefficient exploration.

2. High variance and slow: since many trials give no gradient, so training is noisy.

3. Easy to game: models learn surface patterns of format without better math.

4. Poor transfer: gains do not generalize to new prompts or longer chains.

**Possible solutions**

1. Softer answer rewards.  
Use a proximity score $r_{\text{num}} = \exp\!\big(-|y - \hat{y}| / \tau\big)$ or a scaled error after canonicalizing numeric forms.

2. Smooth format reward.
Score the final line with edit distance, for example:
$$r_{\text{fmt}} = 1 - \min\!\left(1,\; \frac{\mathrm{ED}(\text{last\_line}, \langle\text{answer}\rangle \hat{y} \langle/\text{answer}\rangle)}{L}\right)$$
where $\text{ED}$ is the [Levenshtein distance](https://fr.wikipedia.org/wiki/Distance_de_Levenshtein), and $L$ is a normalizing constant to stay in $[0, 1]$

3. Stepwise reasoning.  
Verify operations and set $r_{\text{steps}} = \frac{\#\text{verified}}{\#\text{parsed}}$  
**Example :**  
**From** $7\times 8=56$
   - $A = 7\times 8$
   - $B = 56$
   - Evaluate: $A=56$, $B=56$ -> correct  
   
   **From** $56+5=56$
   - $A = 56+5$
   - $B = 56$  
   - Evaluate: $A=61$, $B=65$ -> wrong

   So, $r_\text{steps}= \frac{1}{2}$


Thus, we can combine those rewards and get :   
$
r = 0.4\,r_{\text{num}} + 0.3\,r_{\text{fmt}} + 0.3\,r_{\text{steps}}$ and $ r = \text{clip}(r, 0, 1)$

